In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# !pip install gdown
!pip install textstat
# !pip install spacy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.1/105.1 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 8.0 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import json
from tqdm import tqdm
import textstat
import spacy

def JsonToDataFrame(filePath):
  texts = []
  labels = []
  models = []
  ids = []
  sources = []


  dct = {'text': texts, 'model': models,"label":labels,"source":sources, "id":ids }
  with open(filePath, "r") as f:
    for line in f:
      curr = json.loads(line)
      texts.append(curr['text'])
      labels.append(curr['label'])
      models.append(curr['model'])
      sources.append(curr['source'])
      ids.append(curr['id'])


  df = pd.DataFrame(dct)
  return df

def data_stats(df):
  print("[INFO]: Unique AI models in dataset: ", df['model'].unique())
  print("[INFO]: Unique AI models in dataset: ", df['label'].unique())

def save_to_csv(df, outputPATH):
  df.to_csv("SubtaskA_dev_Monolingual.csv", index = False)


def get_corpuses(data):
  ai_corpus = set()
  human_corpus = set()

  for i in tqdm(range(len(data))):
    text = data['text'].iloc[i]
    if data['label'].iloc[i] ==0:
      for t in text.split(" "):
        human_corpus.add(t)
    else:
      for t in text.split(" "):
        ai_corpus.add(t)

  return ai_corpus, human_corpus


def complexity_of_sentence(sent):
  ## score between 1 to 100 (100 easiest , 1 hardest)
  return textstat.flesch_reading_ease(sent)

def length_of_sentence(sent, nlp):
  tokens = nlp(sent)
  return len(tokens)

# def topic_modeling(sent):
#   pass

def spelling_errors(sent):
  pass

def emoji(sent):
  pass


In [ ]:
train_data = pd.read_csv("/content/drive/MyDrive/SharedTask/SubtaskA_train_Monolingual.csv")
train_data.head()

,text,model,label,source,id
0,Forza Motorsport is a popular racing game that...,chatGPT,1,wikihow,0
1,Buying Virtual Console games for your Nintendo...,chatGPT,1,wikihow,1
2,Windows NT 4.0 was a popular operating system ...,chatGPT,1,wikihow,2
3,How to Make Perfume\n\nPerfume is a great way ...,chatGPT,1,wikihow,3
4,How to Convert Song Lyrics to a Song'\n\nConve...,chatGPT,1,wikihow,4


Shuffle the data

In [ ]:
train_data = train_data.sample(frac = 1)

In [ ]:
train_data = train_data[:50000]

In [ ]:
X = train_data['text']
y = train_data['label']

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

In [ ]:
from sklearn.linear_model import LogisticRegression
# from sklearn.svm import SVC
# from sklearn.naive_bayes import MultinomialNB
# from sklearn.tree import DecisionTreeClassifier
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.ensemble import ExtraTreesClassifier
# from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
# from sklearn.ensemble import GradientBoostingClassifier

In [ ]:
lg = LogisticRegression(penalty='l1',solver='liblinear')
# sv = SVC(kernel='sigmoid',gamma=1.0)
# mnb = MultinomialNB()
# dtc = DecisionTreeClassifier(max_depth=5)
# knn = KNeighborsClassifier()
# rfc = RandomForestClassifier(n_estimators=50,random_state=2)
# etc = ExtraTreesClassifier(n_estimators=50,random_state=2)
# abc = AdaBoostClassifier(n_estimators=50,random_state=2)
bg = BaggingClassifier(n_estimators=50,random_state=2)
# gbc = GradientBoostingClassifier(n_estimators=50,random_state=2)

In [ ]:
import sklearn.metrics as metrics
import pickle

def prediction(model,X_train,X_test,y_train,y_test, model_name):
    model.fit(X_train,y_train)
    pr = model.predict(X_test)
    pickle.dump(model, open('/content/drive/MyDrive/SharedTask/Models/' + model_name + '.pkl', 'wb'))
    acc_score = metrics.accuracy_score(y_test,pr)
    f1= metrics.f1_score(y_test,pr,average="binary", pos_label=1)
    return acc_score,f1

acc_score = {}
f1_score={}
clfs= {
    'LR':lg,
    'BG':bg
}
# for name,clf in clfs.items():
#     acc_score[name],f1_score[name]= prediction(clf,X_train_tfidf,X_test_tfidf,y_train,y_test, name)

#     #View those scores
#     print(acc_score)
#     print(f1_score)

## Dev Data test

In [ ]:
dev_data = pd.read_csv("/content/drive/MyDrive/SharedTask/SubtaskA_dev_Monolingual.csv")
dev_data.head()

,text,model,label,source,id
0,Giving gifts should always be enjoyable. Howe...,bloomz,1,wikihow,0
1,Yveltal (Japanese: ユベルタル) is one of the main a...,bloomz,1,wikihow,1
2,If you'd rather not annoy others by being rude...,bloomz,1,wikihow,2
3,If you're interested in visiting gravesite(s) ...,bloomz,1,wikihow,3
4,The following are some tips for becoming succe...,bloomz,1,wikihow,4


Logistic Regression

In [ ]:
dev_tfidf = vectorizer.transform(dev_data['text'])
lg.fit(X_train_tfidf,y_train)
metrics.accuracy_score(lg.predict(dev_tfidf), dev_data['label'])

0.5712

Bagging Classifier

In [ ]:
bg.fit(X_train_tfidf,y_train)
metrics.accuracy_score(bg.predict(dev_tfidf), dev_data['label'])

In [ ]:
print("Number of words that are in AI corpus but not in Human Corpus: ", len(ai_corpus.difference(human_corpus)))
print("Number of words in AI corpus: ",len(ai_corpus))
print("Number of words in Human corpus: ",len(human_corpus))

581888

2034477